In [ ]:
#!/usr/bin/python3
# coding: utf-8
# Tokyo

In [ ]:
import codecs
from datetime import datetime as dt
from datetime import timedelta as td
from jma_csvdl import save_jma_data, parse_jma_csv
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
from plotly.subplots import make_subplots
import sys
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    show_and_save_plotly, make_exp_fit_graph
from urllib.request import urlretrieve

In [ ]:
today_str = dt.now().isoformat()[:16].replace('T', ' ')
# 東京都の OpenData を参照する
patients_file = "130001_tokyo_covid19_patients.csv"
patients_uri = "https://stopcovid19.metro.tokyo.lg.jp/data/"
download_if_needed(patients_uri, patients_file)

In [ ]:
patients = []
# CSVデータを整形する
with codecs.open(patients_file, encoding="utf-8") as f:
    l = f.readline()
    while l:
        l = f.readline().replace("\r\n", "")
        arr = l.split(',')
        if len(arr) == 17:
            patients.append(
                [
                    code2int(arr[0]), # No
                    code2int(arr[1]), # code
                    arr[2], # state
                    arr[4], # date
                    arr[5], # day of week
                    arr[7], # live in
                    age2int(arr[8]), # age
                    arr[9], # sex
                    arr[16] # discharge
                ]
            )

print("Total: {}".format(len(patients)))
# 先頭３つのデータは飛ばす
patients = patients[3:]
#print("Total: {}".format(len(patients)))

In [ ]:
# Pandas DataFrame を作成する
df = pd.DataFrame(patients, columns=['No', 'Code', 'State', 'Date', 'DoW', 'LiveIn', 'Age', 'Sex', 'Discharge'])

In [ ]:
# Daily の新規感染者を集計する
daily_new = df.groupby('Date').size()
print("Daily New: {}".format(daily_new[-1]))
# duration
frm_date = daily_new.index[0]
end_date = daily_new.index[-1]
print("From: {} To: {}".format(frm_date, end_date))

with open("tokyo.prev.tmp", "rt") as f:
    prev = f.read().rstrip()
if end_date == prev:
    print("maybe the same data, nothing to do.")
    if "ipy" in sys.argv[0]:
        pass#exit()
    else:
        sys.exit()
with open("tokyo.prev.tmp", "wt") as f:
    f.write(end_date)

In [ ]:
# moving average by week
ave_mov_days = 7
# 移動平均を算出する
mov_mean = daily_new.rolling(ave_mov_days).mean()

In [ ]:
# 4 weeks later
xbins = [dt.strptime(x, "%Y-%m-%d") for x in daily_new.index]
days2pred = 2 * ave_mov_days # 2 weeks
# 2週間先の日付列を取得する
two_weeks_later = pd.date_range(xbins[-1] + td(days=1), xbins[-1] + td(days=days2pred)).to_pydatetime()
xbins_pred = xbins.copy()
xbins_pred.extend(two_weeks_later)

X = np.arange(0, len(daily_new.index.values))[:, np.newaxis]
X_pred = np.arange(0, len(xbins_pred))[:, np.newaxis]
y_gpr = get_gpr_predict(X, daily_new.values, X_pred, 80, 10, 200)

In [ ]:
save_jma_data("tokyo-jma.csv", city_code="s47662")
weather_tokyo = parse_jma_csv("tokyo-jma.csv")
print("Loaded {} data.".format(len(weather_tokyo)))
df_weather = pd.DataFrame(weather_tokyo, columns=['Date', 'Temp', 'RH', 'VP', 'AP', 'AH', 'Fd'])
fig = px.scatter(df_weather, x='Date', y=['AH'])
fig.update_layout(template='plotly_dark')
if "ipy" in sys.argv[0]:
    fig.show()
np_weather = np.array(weather_tokyo)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=xbins, y=daily_new, mode='markers', name='新規',
               marker=dict(size=4)), secondary_y=False)
fig.add_trace(go.Bar(
    x=xbins, y=mov_mean, name='7日移動平均', opacity=0.6), 
              secondary_y=False)
fig.add_trace(
    go.Scatter(x=xbins_pred, y=y_gpr, mode='lines', name='予測',
               line=dict(width=1)), secondary_y=False)
fig.add_trace(go.Scatter(
    x=np_weather[:, 0], # 日付
    y=np_weather[:, 5], # 絶対湿度
    name="絶対湿度",
    line=dict(width=1)), secondary_y=True)
fig.update_layout(
    xaxis=dict(title='日付', type='date',
               dtick=1209600000.0, tickformat="%_m/%-d",
               range=[xbins[10], xbins_pred[-1]]
              ),
    yaxis=dict(title='人数'),
    yaxis2=dict(title='東京平均容積絶対湿度 [g/㎥]'),
    title='東京都 新型コロナ 新規感染者数/絶対湿度({})'.format(today_str),
)
show_and_save_plotly(fig, "tokyo.jpg", js=False)

In [ ]:
if False:
    fig, ax = create_basic_plot_figure()
    ymax = max(daily_new)
    plt.title('東京都 新型コロナ予測 新規感染者数', fontname=FONT_NAME)
    plt.xlim(xbins_pred[ave_mov_days], xbins_pred[-1])
    plt.ylim(0, ymax)
    plt.plot(xbins, daily_new,  '.', linewidth=1, color='#c0ffc0')
    plt.bar(xbins, mov_mean, color='#008080', edgecolor='#008080', alpha=1)
    plt.legend(labels=["新規", "７日移動平均"], loc='upper left', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    ax2 = ax.twinx()
    plt.plot(xbins_pred, y_gpr, '-', linewidth=1.5, color='#ff8080')
    plt.ylim(0, ymax)
    plt.legend(labels=["ガウス過程回帰"], loc='upper right', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    show_and_clear(fig, "docs/images/tokyo.jpg")

In [ ]:
tw_body = "東京都 新型コロナ予測 新規感染者数(" + today_str + ")"
tw_body += " https://geneasyura.github.io/cov19-hm/tokyo.html "
tw = get_twitter()
tweet_with_image(tw, "docs/images/tokyo.jpg", tw_body)

In [ ]:
# Github JSON データ
github_uri = "https://raw.githubusercontent.com/tokyo-metropolitan-gov/covid19/development/data/"
raw_files = dict(
    pos_rate = "positive_rate.json", # 陽性率
    pos_detail = "daily_positive_detail.json", # 経路情報
    patient = "patient.json", # 区別
)

for k, v in raw_files.items():
    download_if_needed(github_uri, v)

In [ ]:
keys_rate = [
    #"diagnosed_date",
    "positive_count",
    "negative_count",
    "positive_rate",
    "weekly_average_diagnosed_count",
    "pcr_positive_count",
    "pcr_negative_count",
    "antigen_positive_count",
    "antigen_negative_count"
]

In [ ]:
pos_rate_np = json2nparr(keys_rate, raw_files['pos_rate'])
# 検査件数
tests_cnt = pos_rate_np[:, 1] + pos_rate_np[:, 2]

In [ ]:
xbins = pos_rate_np[:, 0]
two_weeks_later = pd.date_range(xbins[-1] + td(days=1), xbins[-1] + td(days=days2pred)).to_pydatetime()
xbins_pred = xbins.tolist()
xbins_pred.extend(two_weeks_later)
X = np.arange(0, len(pos_rate_np[:, 0]))[:, np.newaxis]
X_pred = np.arange(0, len(xbins_pred))[:, np.newaxis]

y_test = get_gpr_predict(X, tests_cnt, X_pred, 10, 10, 10)
y_rate = get_gpr_predict(X, pos_rate_np[:, 3], X_pred)

In [ ]:
if False:
    fig, ax = create_basic_plot_figure()
    plt.title('東京都 新型コロナ 検査人数/陽性率', fontname=FONT_NAME)
    plt.ylabel('人数', fontname=FONT_NAME)
    plt.ylim(0, max(tests_cnt)+100)
    plt.plot(xbins, tests_cnt, '.', linewidth=1.5, color='#00ff00')
    plt.plot(xbins_pred, y_test, '-', color='#00ff00', linewidth=1)
    plt.bar(xbins, pos_rate_np[:, 4], color='#008000', edgecolor='#008000', alpha=.75)
    plt.legend(labels=["検査人数", "予測値", "移動平均"], loc='upper left', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    ax2 = ax.twinx()
    plt.ylim(0, 50)
    ax2.plot(xbins_pred, y_rate, '-', color='pink', linewidth=1)
    ax2.bar(xbins, pos_rate_np[:, 3], color='pink', edgecolor='pink', alpha=.5)
    plt.legend(labels=["予測値", "陽性率[%]"], loc='upper right', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    show_and_clear(fig, "docs/images/tokyo-rate.jpg")

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=xbins, y=tests_cnt, mode='markers', name='検査人数',
                                       marker=dict(size=4)), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=pos_rate_np[:, 4], name='移動平均', opacity=0.5), 
                    secondary_y=False)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_test, mode='lines', name='予測値',
                         line=dict(width=1)), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=pos_rate_np[:, 3], name="陽性率[%]", opacity=0.5),
                         secondary_y=True)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_rate, name="予測値",
                         line=dict(width=1)), secondary_y=True)
fig.update_layout(
    xaxis=dict(title='日付', type='date', 
               dtick=1209600000.0, tickformat="%_m/%-d",
               range=(xbins_pred[7], xbins_pred[-1])),
    yaxis=dict(title='人数'),
    title='東京都 新型コロナ 検査人数/陽性率({})'.format(today_str),
)
show_and_save_plotly(fig, "tokyo-rate.jpg", js=False)

In [ ]:
today_str = dt.now().isoformat()[:19].replace('T', ' ')
tw_body = "東京都 新型コロナ予測 検査人数/陽性率(" + today_str + ") "
tw_body += " https://geneasyura.github.io/cov19-hm/tokyo.html "
tweet_with_image(tw, "docs/images/tokyo-rate.jpg", tw_body)

In [ ]:
keys_detail = [
    #"diagnosed_date",
    "count",
    "missing_count",
    "reported_count",
    "weekly_gain_ratio",
    "untracked_percent",
    "weekly_average_count",
    "weekly_average_untracked_count",
    "weekly_average_untracked_increse_percent"]

In [ ]:
pos_detail_np = json2nparr(keys_detail, raw_files['pos_detail'])
xbins = pos_detail_np[:, 0]

In [ ]:
two_weeks_later = pd.date_range(xbins[-1] + td(days=1), xbins[-1] + td(days=days2pred)).to_pydatetime()
xbins_pred = xbins.tolist()
xbins_pred.extend(two_weeks_later)
X = np.arange(0, len(pos_detail_np[:, 0]))[:, np.newaxis]
X_pred = np.arange(0, len(xbins_pred))[:, np.newaxis]

In [ ]:
y_un_rate = get_gpr_predict(X, pos_detail_np[:, 5], X_pred, 1.0, 1.0, 1.0)
y_cases = get_gpr_predict(X, pos_detail_np[:, 1], X_pred)
y_untrack = get_gpr_predict(X, pos_detail_np[:, 2], X_pred)

In [ ]:
if False:
    fig, ax = create_basic_plot_figure()
    plt.title('東京都 新型コロナ予測 経路不明率', fontname=FONT_NAME)
    plt.ylim(0, 100)
    plt.ylabel('経路不明率[%]', fontname=FONT_NAME)
    plt.plot(xbins_pred, y_un_rate, '-', color='#ff8080', linewidth=1)
    plt.bar(xbins, pos_detail_np[:, 5], color='#c08080', edgecolor='#c08080', alpha=.8)
    plt.legend(labels=["予測値x", "経路不明率[%]"], loc='upper left', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    ax2 = ax.twinx()
    plt.ylim(0, np.max(pos_detail_np[:, 1])+10)
    plt.ylabel('人数', fontname=FONT_NAME)
    ax2.plot(xbins, pos_detail_np[:, 1], '.', color='#00ff00', label='感染者')
    ax2.plot(xbins_pred, y_cases, '-', color='#00ff00', linewidth=1, label='予測値')
    ax2.plot(xbins, pos_detail_np[:, 2], '.', color='white', label="経路不明者")
    ax2.plot(xbins_pred, y_untrack, '-', color='white', linewidth=1, label='予測値')
    ax2.bar(xbins, moving_average(pos_detail_np[:, 1]), color='#00c000', edgecolor='#00c000', alpha=.5, label="感染者-移動平均")
    ax2.bar(xbins, pos_detail_np[:, 7], color='#000080', edgecolor='#000080', alpha=.3, label="経路不明者-移動平均")
    plt.legend(loc='upper right', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    show_and_clear(fig, "docs/images/tokyo-track.jpg")

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=xbins, y=pos_detail_np[:, 5], name='経路不明率[%]', opacity=0.3, marker_color='#c08080'),  secondary_y=True)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_un_rate, name='経路不明率-予測値', line=dict(width=1)), secondary_y=True)
fig.add_trace(go.Scatter(x=xbins, y=pos_detail_np[:, 1], mode='markers', name='感染者', marker=dict(size=4, color='#00ff00')), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=moving_average(pos_detail_np[:, 1]), name='感染者-移動平均', opacity=0.6, marker_color='#00c000'),  secondary_y=False)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_cases, name="感染者-予測値",   line=dict(width=1)), secondary_y=False)
fig.add_trace(go.Scatter(x=xbins, y=pos_detail_np[:, 2], mode='markers', name='経路不明者', marker=dict(size=4, color='#ffffff')), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=pos_detail_np[:, 7], name='経路不明者-移動平均', opacity=0.7, marker_color='#000080'),  secondary_y=False)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_untrack, name="経路不明者-予測値",   line=dict(width=1)), secondary_y=False)
fig.update_layout(
    xaxis=dict(title='日付', type='date',
               dtick=1209600000.0, tickformat="%_m/%-d",
               range=[xbins[10], xbins_pred[-1]]
              ),
    yaxis=dict(title='人数'),
    title='東京都 新型コロナ 経路不明率({})'.format(today_str),
)
show_and_save_plotly(fig, "tokyo-track.jpg", js=False)

In [ ]:
today_str = dt.now().isoformat()[:19].replace('T', ' ')
tw_body = "東京都 新型コロナ予測 経路不明率(" + today_str + ") "
tw_body += " https://geneasyura.github.io/cov19-hm/tokyo.html "
tweet_with_image(tw, "docs/images/tokyo-track.jpg", tw_body)

In [ ]:
title = '東京都 新型コロナ 新規感染者数/累乗近似 (' + today_str + ')'
xos = 270
make_exp_fit_graph(tw, 
    xbins[xos:], pos_detail_np[xos:, 1], 
    title, "tokyo-fit.jpg", "tokyo-doubling-time.html", "tokyo.html")

In [ ]:
title = '東京都 新型コロナ 新規移動平均/累乗近似 (' + today_str + ')'
ydata = moving_average(pos_detail_np[:, 1])
xos = 250
make_exp_fit_graph(tw, 
    xbins[xos:], ydata[xos:], 
    title, "tokyo-fit-ave.jpg", "tokyo-doubling-time-ave.html", "tokyo.html")